In [1]:
import pandas as pd
import random
import numpy as np
import networkx as nx
import nltk
import tensorflow as tf
import collections
import math
import os.path
from six.moves import xrange
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.cluster import MiniBatchKMeans
from sklearn.cluster import Birch

### Load Document Data

In [2]:
def load():
    ppp = nltk.data.load('../../Downloads/ppp.txt', encoding='utf8')
    words_p = nltk.tokenize.wordpunct_tokenize(ppp)[130:]
    alw = nltk.data.load('../../Downloads/alw.txt', encoding='utf8')
    words_a = nltk.tokenize.wordpunct_tokenize(alw)[143:]
    return words_a, words_p

### Generate Batches Within W2V

In [3]:
data_index = 0
def gen_batch(data, batch_size, skip_window, num_skips):
    global data_index
    assert batch_size % num_skips == 0
    assert num_skips <= 2 * skip_window
    batch = np.ndarray(shape=(batch_size), dtype=np.int32)
    labels = np.ndarray(shape=(batch_size, 1), dtype=np.int32)
    span = 2 * skip_window + 1 # [ skip_window target skip_window ]
    buffer = collections.deque(maxlen=span)
    for _ in range(span):
        buffer.append(data[data_index])
        data_index = (data_index + 1) % len(data)
    for i in range(batch_size // num_skips):
        target = skip_window  # target label at the center of the buffer
        targets_to_avoid = [ skip_window ]
        for j in range(num_skips):
            while target in targets_to_avoid:
                target = random.randint(0, span - 1)
            targets_to_avoid.append(target)
            batch[i * num_skips + j] = buffer[skip_window]
            labels[i * num_skips + j, 0] = buffer[target]
        buffer.append(data[data_index])
        data_index = (data_index + 1) % len(data)
    return batch, labels


### Build Vocab

In [4]:
def build_vocab(words, vocabulary_size):
    count = [['UNK', -1]]
    count.extend(collections.Counter(words).most_common(vocabulary_size - 1))
    dictionary = dict()
    for word, _ in count:
        dictionary[word] = len(dictionary)
    data = list()
    unk_count = 0
    for word in words:
        if word in dictionary:
            index = dictionary[word]
        else:
            index = 0  # dictionary['UNK']
            unk_count += 1
        data.append(index)
    count[0][1] = unk_count
    reverse_dictionary = dict(zip(dictionary.values(), dictionary.keys()))
    return data, dictionary, reverse_dictionary

# We're Gonna Start By Just Saving Dicts

### Word2Vec TensorFlow

In [5]:
def W2V(batch_size, embedding_size, skip_window, num_skips, valid_size,
       valid_window, valid_examples, num_sampled, vocabulary_size,
       num_steps, data, revdic):
    graph = tf.Graph()
    
    with graph.as_default():
        train_inputs = tf.placeholder(tf.int32, shape=[batch_size])
        train_labels = tf.placeholder(tf.int32, shape=[batch_size, 1])
        valid_dataset = tf.constant(valid_examples, dtype=tf.int32)
        
        with tf.device('/cpu:0'):
            
            embeddings = tf.Variable(
                            tf.random_uniform([vocabulary_size, embedding_size], -1.0, 1.0))
            embed = tf.nn.embedding_lookup(embeddings, train_inputs)
        
        nce_weights = tf.Variable(
            tf.truncated_normal([vocabulary_size, embedding_size], 
                               stddev=1.0 / math.sqrt(embedding_size)))
        nce_biases = tf.Variable(tf.zeros([vocabulary_size]))
        
        loss = tf.reduce_mean(
            tf.nn.nce_loss(nce_weights, nce_biases, embed, train_labels,
                          num_sampled, vocabulary_size))
        optimizer = tf.train.GradientDescentOptimizer(1.0).minimize(loss)
        
        norm = tf.sqrt(tf.reduce_sum(tf.square(embeddings), 
                                     1, keep_dims=True))
        normalized_embeddings = embeddings / norm
        valid_embeddings = tf.nn.embedding_lookup(
            normalized_embeddings, valid_dataset)
        similarity = tf.matmul(
            valid_embeddings, normalized_embeddings, transpose_b=True)
        
        init = tf.initialize_all_variables()
    
    with tf.Session(graph=graph) as session:
        
        init.run()
        print("Initialized")
        #saver = tf.train.Saver({'In'})
        
        average_loss = 0
        for step in xrange(num_steps):
            batch_inputs, batch_labels = gen_batch(
                data, batch_size, skip_window, num_skips)
            feed_dict = {train_inputs : batch_inputs, 
                         train_labels : batch_labels}
            
            _, loss_val = session.run([optimizer, loss], feed_dict=feed_dict)
            average_loss += loss_val

        
            if step % 2000 == 0:
                if step > 0:
                    average_loss /= 2000
                # The average loss is an estimate of the loss over the last 2000 batches.
                print("Average loss at step ", step, ": ", average_loss)
                average_loss = 0

            # Note that this is expensive (~20% slowdown if computed every 500 steps)
            if step % 10000 == 0 and step > 0:
                sim = similarity.eval()
                for i in xrange(valid_size):
                    valid_word = revdic[valid_examples[i]]
                    top_k = 8 # number of nearest neighbors
                    nearest = (-sim[i, :]).argsort()[1:top_k+1]
                    log_str = "Nearest to %s:" % valid_word
                    for k in xrange(top_k):
                        close_word = revdic[nearest[k]]
                        log_str = "%s %s," % (log_str, close_word)
                    print(log_str)

        final_embeddings = normalized_embeddings.eval()
        return final_embeddings

### Word2Vec Parameters

In [5]:
#words = load()
#KB = prep_graph(words)
batch_size = 256
embedding_size = 200  # Dimension of the embedding vector.
skip_window = 3      # How many words to consider left and right.
num_skips = 4         # How many times to reuse an input to generate a label.

# We pick a random validation set to sample nearest neighbors. Here we limit the
# validation samples to the words that have a low numeric ID, which by
# construction are also the most frequent.
valid_size = 10    # Random set of words to evaluate similarity on.
valid_window = 200  # Only pick dev samples in the head of the distribution.
num_sampled = 100    # Number of negative examples to sample.
num_steps = 20000

## Classic Example

In [7]:
def classic(l_words, t_words, vocab_size):
    data_l, dictionary_l, reverse_dictionary_l = build_vocab(l_words, 
                                                             vocab_size)
    
    valid_examples = np.random.choice(valid_window, 
                                      valid_size, replace=False)
    
    embeddings_l = W2V(batch_size, embedding_size, skip_window,
            num_skips, valid_size, valid_window,
            valid_examples, num_sampled, vocab_size,
            num_steps, data_l, reverse_dictionary_l)
    
    data_index = 0
    
    data_t = [dictionary_l[word_t] 
              if word_t in dictionary_l else dictionary_l['UNK']
              for word_t in t_words]
    
    
    valid_examples = np.random.choice(valid_window, 
                                      valid_size, replace=False)
    
    embeddings_t = W2V(batch_size, embedding_size, skip_window,
            num_skips, valid_size, valid_window,
            valid_examples, num_sampled, vocab_size,
            num_steps, data_l, reverse_dictionary_l)


### Make a Graph Where Nodes are Words with Number Attributes

In [6]:
def prep_graph(words):
    KB2 = nx.Graph()
    for word in words:
        if not KB2.has_node(word):
            KB2.add_node(word)
        
    KB3 = nx.convert_node_labels_to_integers(KB2, label_attribute='word')
    KB4 = nx.Graph()
    
    for node in KB3.nodes(True):
        KB4.add_node(node[1]['word'], number=node[0])
    return KB4

### We call this each time we want to add new layers

In [9]:
def add_level(words, embeddings, KB, n_cluster):
    for index, word in enumerate(words[:-1]):
        if KB.has_node(word) and KB.has_node(words[index+1]):
            if KB.has_edge(word, words[index+1]):
                node_name = KB.edge[word][words[index+1]]['node']
                words = np.insert(words, index+1, str(node_name))
    
    
    data = [KB.node[word]['number'] for word in words if word in KB.node]
    embed_data = np.array([embeddings[wordnum] for wordnum in data])
    next_lvl_raw = embed_data[1:] - embed_data[:-1]
    mbatch = MiniBatchKMeans(n_clusters=n_cluster, 
                             batch_size=max(len(words)*.05, n_cluster+1), 
                             max_iter=100000)
    next_lvl_cent = mbatch.fit(embed_data)

    vocab_size = KB.number_of_nodes()
    
    for num in range(vocab_size, vocab_size+n_cluster):
        KB.add_node(str(num), number=num)
    
    words_n = np.array([words[0]])
    for i in range(1, len(words)-1):
        t = next_lvl_cent.labels_[i-1]
        words_n = np.append(words_n, [str(t+vocab_size), words[i+1]])
        KB.add_edge(words[i], words[i+1], node=str(t+vocab_size))
        
    return words_n, KB     

## Our Method
We train twice but since we start from scratch each time, it doesn't really matter

In [10]:
def new(words_l, words_t, vocab_size):
    global data_index
    data_index = 0
    m_common = [item[0] for item in collections.Counter(words_l).most_common(n=500)]
    words_l = np.array([word for word in words_l if word in m_common])
    KB = prep_graph(words_l)
    
    valid_examples = np.random.choice(valid_window, 
                                      valid_size, replace=False)
    for i in range(4):
        if i > 0:
            words_l, KB = add_level(words_l, embeddings_l, KB, 500)
        data_index = 0
        vocab_size = KB.number_of_nodes()
        revdic = {node[1]['number']: node[0] for node in KB.nodes(True)}
        data_l = [KB.node[word]['number'] for word in words_l]
        embeddings_l = W2V(batch_size, embedding_size, skip_window,
                num_skips, valid_size, valid_window, valid_examples, 
                       num_sampled, vocab_size, 10000, data_l, revdic)
    
    
    words_t = np.array([word for word in words_t if word in m_common])
    for index, word in enumerate(words_t[:-1]):
        if KB.has_node(word) and KB.has_node(words_t[index+1]):
            if KB.has_edge(word, words_t[index+1]):
                node_name = KB.edge[word][words_t[index+1]]['node']
                words_t = np.insert(words_t, index+1, str(node_name))
    
    data_index = 0
    data_t = [KB.node[word]['number'] for word in words_t if word in KB.node]
    embeddings_t = W2V(batch_size, embedding_size, skip_window,
                num_skips, valid_size, valid_window, valid_examples, 
                       num_sampled, vocab_size, num_steps, data_t, revdic)
    

## Actual Test Area

In [6]:
words_l, words_t = load()

In [7]:
vocab_size = 2000

In [32]:
classic(words_l, words_t, vocab_size)

Initialized
Average loss at step  0 :  234.604553223
Average loss at step  2000 :  11.0108101764
Average loss at step  4000 :  5.10339176452
Average loss at step  6000 :  4.99395320916
Average loss at step  8000 :  4.91171428978
Average loss at step  10000 :  4.84941917634
Nearest to heard: whisper, off, sending, just, Have, CHORUS, Footman, tears,
Nearest to get: walked, archbishop, Ugh, LITTLE, *, stoop, ve, but,
Nearest to ve: Only, 1, smile, kind, Soup, All, distribution, finished,
Nearest to t: *, elbow, haven, washing, milk, tried, hurriedly, don,
Nearest to by: none, However, cards, chimney, Duchess, lying, ), shrill,
Nearest to electronic: grand, MUST, WILLIAM, fashion, Please, adventures, U, changed,
Nearest to !’: ?’, screamed, change, provisions, SAID, feared, Exactly, On,
Nearest to things: cautiously, sort, somebody, thimble, prisoner, barrowful, want, PROJECT,
Nearest to or: dear, Wake, 11, shaking, crimson, OR, grow, watched,
Nearest to UNK: longer, severely, beg, twist,

In [116]:
new(words_l, words_t, vocab_size)

Initialized
Average loss at step  0 :  153.989837646
Average loss at step  2000 :  5.16240739226
Average loss at step  4000 :  4.09049435699
Average loss at step  6000 :  4.02145858324
Average loss at step  8000 :  3.98238668633


/home/lenny/anaconda3/lib/python3.5/site-packages/sklearn/cluster/k_means_.py:1349: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
  distances = np.zeros(self.batch_size, dtype=X.dtype)
/home/lenny/anaconda3/lib/python3.5/site-packages/sklearn/cluster/k_means_.py:1360: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
  validation_indices = random_state.randint(0, n_samples, init_size)
/home/lenny/anaconda3/lib/python3.5/site-packages/sklearn/cluster/k_means_.py:671: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
  init_indices = random_state.randint(0, n_samples, init_size)
/home/lenny/anaconda3/lib/python3.5/site-packages/sklearn/cluster/k_means_.py:1409: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
  0, n_samples, self.b

Initialized
Average loss at step  0 :  193.846221924
Average loss at step  2000 :  5.93933841205
Average loss at step  4000 :  3.50263933849
Average loss at step  6000 :  3.40718356699
Average loss at step  8000 :  3.34587858325
Initialized
Average loss at step  0 :  222.103393555
Average loss at step  2000 :  7.31874467373
Average loss at step  4000 :  3.10586724973
Average loss at step  6000 :  2.96929880857
Average loss at step  8000 :  2.93134997511
Initialized
Average loss at step  0 :  236.08770752
Average loss at step  2000 :  8.83436334783
Average loss at step  4000 :  3.03062243599
Average loss at step  6000 :  2.81695212293
Average loss at step  8000 :  2.74504919231
Initialized
Average loss at step  0 :  237.448974609
Average loss at step  2000 :  8.09999686986
Average loss at step  4000 :  3.64540397012
Average loss at step  6000 :  3.73201282293
Average loss at step  8000 :  3.66827483433
Average loss at step  10000 :  3.45715198344
Nearest to size: air, as, 503, 662, 723,

# Now We Test For Maintaining State of W2V

In [7]:
def W2V2(batch_size, embedding_size, skip_window, num_skips, valid_size,
       valid_window, valid_examples, num_sampled, vocabulary_size,
       num_steps, data, revdic):
    graph = tf.Graph()
    
    with graph.as_default():
        def weight_summary(var, name):
          """Attach a lot of summaries to a Tensor."""
          with tf.name_scope('summaries'):
            mean = tf.reduce_mean(var)
            tf.scalar_summary('mean/' + name, mean)
            with tf.name_scope('stddev'):
                stddev = tf.sqrt(tf.reduce_mean(tf.square(var - mean)))
            tf.scalar_summary('stddev/' + name, stddev)
            tf.scalar_summary('max/' + name, tf.reduce_max(var))
            tf.scalar_summary('min/' + name, tf.reduce_min(var))
            tf.histogram_summary(name, var)
        def scalar_summary(var, name):
            with tf.name_scope('summaries'):
                tf.scalar_summary('scalar/'+name, var)
                
        train_inputs = tf.placeholder(tf.int32, shape=[batch_size])
        train_labels = tf.placeholder(tf.int32, shape=[batch_size, 1])
        valid_dataset = tf.constant(valid_examples, dtype=tf.int32)

        with tf.device('/cpu:0'):
            
            embeddings = tf.Variable(
                            tf.random_uniform([vocabulary_size, embedding_size], -1.0, 1.0), name="emb")
            weight_summary(embeddings, 'embeddings')
            embed = tf.nn.embedding_lookup(embeddings, train_inputs)
        
        nce_weights = tf.Variable(
            tf.truncated_normal([vocabulary_size, embedding_size], 
                               stddev=1.0 / math.sqrt(embedding_size)), name="nw")
        weight_summary(nce_weights, 'nce_weights')
        nce_biases = tf.Variable(tf.zeros([vocabulary_size]), name="nb")
        weight_summary(nce_biases, 'nce_biases')
        
        loss = tf.reduce_mean(
            tf.nn.nce_loss(nce_weights, nce_biases, embed, train_labels,
                          num_sampled, vocabulary_size))
        #scalar_summary(loss, 'loss')
        optimizer = tf.train.GradientDescentOptimizer(1.0).minimize(loss)
        
        norm = tf.sqrt(tf.reduce_sum(tf.square(embeddings), 
                                     1, keep_dims=True))
        #scalar_summary(norm, 'norm')
        
        normalized_embeddings = embeddings / norm
        weight_summary(normalized_embeddings, 'normalized_embeddings')
        valid_embeddings = tf.nn.embedding_lookup(
            normalized_embeddings, valid_dataset)
        weight_summary(valid_embeddings, 'valid_embeddings')
        similarity = tf.matmul(
            valid_embeddings, normalized_embeddings, transpose_b=True)
        weight_summary(similarity, 'similarity')
        
        merged = tf.merge_all_summaries()
        
        init = tf.initialize_all_variables()
    
    
    with tf.Session(graph=graph) as session:
        
        
        saver = tf.train.Saver()
        train_writer = tf.train.SummaryWriter('./summaries' + '/train',
                                      session.graph)
        
        init.run()
        print("Initialized")      
        
        #saver = tf.train.Saver({'In'})
        
        average_loss = 0
        for step in xrange(num_steps):
            batch_inputs, batch_labels = gen_batch(
                data, batch_size, skip_window, num_skips)
            feed_dict = {train_inputs : batch_inputs, 
                         train_labels : batch_labels}
            
            if step % 10 == 0:
                
                _, loss_val, summary = session.run([optimizer, loss, merged], feed_dict=feed_dict)
                average_loss += loss_val
            
                train_writer.add_summary(summary, step)
            else:
                _, loss_val = session.run([optimizer, loss], feed_dict=feed_dict)
                average_loss += loss_val
        
            if step % 2000 == 0:
                if step > 0:
                    average_loss /= 2000
                # The average loss is an estimate of the loss over the last 2000 batches.
                print("Average loss at step ", step, ": ", average_loss)
                average_loss = 0

            # Note that this is expensive (~20% slowdown if computed every 500 steps)
            if step % 10000 == 0 and step > 0:
                sim = similarity.eval()
                for i in xrange(valid_size):
                    valid_word = revdic[valid_examples[i]]
                    top_k = 8 # number of nearest neighbors
                    nearest = (-sim[i, :]).argsort()[1:top_k+1]
                    log_str = "Nearest to %s:" % valid_word
                    for k in xrange(top_k):
                        close_word = revdic[nearest[k]]
                        log_str = "%s %s," % (log_str, close_word)
                    print(log_str)
        final_embeddings = normalized_embeddings.eval()
        return final_embeddings

### Adding Levels

In [8]:
def add_level2(words, embeddings, KB, n_cluster, fit=None):
    words
    for index, word in enumerate(words[:-1]):
        if KB.has_node(word) and KB.has_node(words[index+1]):
            if KB.has_edge(word, words[index+1]):
                node_name = KB.edge[word][words[index+1]]['node']
                words = np.insert(words, index+1, str(node_name))
    
    
    data = [KB.node[word]['number'] for word in words if word in KB.node]
    embed_data = np.array([embeddings[wordnum] for wordnum in data])
    next_lvl_raw = embed_data[1:] - embed_data[:-1]
    
    if not fit:
        mbatch = Birch(n_clusters=n_cluster, branching_factor=200)
        b_tree = mbatch.fit(embed_data)
    else:
        fit.set_params(n_clusters=fit.n_clusters+n_cluster)
        next_lvl_cent = fit.partial_fit(embed_data)
    vocab_size = KB.number_of_nodes()
    
    for num in range(vocab_size, vocab_size+n_cluster):
        KB.add_node(str(num), number=num)
    
    words_n = np.array([words[0]])
    for i in range(1, len(words)-1):
        t = next_lvl_cent.labels_[i-1]
        words_n = np.append(words_n, [str(t+vocab_size), words[i+1]])
        KB.add_edge(words[i], words[i+1], node=str(t+vocab_size))
        
    return words_n, KB, next_lvl_cent     

## Classic - Maintained

In [9]:
def classic(l_words, t_words, vocab_size):
    data_l, dictionary_l, reverse_dictionary_l = build_vocab(l_words, 
                                                             vocab_size)
    
    valid_examples = np.random.choice(valid_window, 
                                      valid_size, replace=False)
    
    embeddings_l = W2V(batch_size, embedding_size, skip_window,
            num_skips, valid_size, valid_window,
            valid_examples, num_sampled, vocab_size,
            num_steps, data_l, reverse_dictionary_l)
    
    data_index = 0
    
    data_t = [dictionary_l[word_t] 
              if word_t in dictionary_l else dictionary_l['UNK']
              for word_t in t_words]
    
    
    valid_examples = np.random.choice(valid_window, 
                                      valid_size, replace=False)
    
    embeddings_t = W2V(batch_size, embedding_size, skip_window,
            num_skips, valid_size, valid_window,
            valid_examples, num_sampled, vocab_size,
            num_steps, data_l, reverse_dictionary_l)


## Our Method - Maintained

In [10]:
def new2(words_l, words_t, vocab_size):
    global data_index
    data_index = 0
    m_common = [item[0] for item in collections.Counter(words_l).most_common(n=500)]
    words_l = np.array([word for word in words_l if word in m_common])
    KB = prep_graph(words_l)
    
    valid_examples = np.random.choice(valid_window, 
                                      valid_size, replace=False)
    for i in range(1):
        if i > 0:
            words_l, KB, fit = add_level2(words_l, embeddings_l, KB, 500)
        data_index = 0
        #vocab_size = KB.number_of_nodes()
        revdic = {node[1]['number']: node[0] for node in KB.nodes(True)}
        data_l = [KB.node[word]['number'] for word in words_l]
        embeddings_l = W2V2(batch_size, embedding_size, skip_window,
                num_skips, valid_size, valid_window, valid_examples, 
                       num_sampled, vocab_size, 10000, data_l, revdic)
    
    
    words_t = np.array([word for word in words_t if word in m_common])
    for index, word in enumerate(words_t[:-1]):
        if KB.has_node(word) and KB.has_node(words_t[index+1]):
            if KB.has_edge(word, words_t[index+1]):
                node_name = KB.edge[word][words_t[index+1]]['node']
                words_t = np.insert(words_t, index+1, str(node_name))
    
    data_index = 0
    data_t = [KB.node[word]['number'] for word in words_t if word in KB.node]
    embeddings_t = W2V2(batch_size, embedding_size, skip_window,
                num_skips, valid_size, valid_window, valid_examples, 
                       num_sampled, vocab_size, num_steps, data_t, revdic)
    

## Actual Test Area

In [11]:
words_l, words_t = load()

In [ ]:
new2(words_l, words_t, 2000)

Initialized
Average loss at step  0 :  250.024047852
Average loss at step  2000 :  8.58146738505
Average loss at step  4000 :  4.14766135931
Average loss at step  6000 :  4.02447207868
Average loss at step  8000 :  3.94582948947


Scratch